In [7]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.util import ngrams

train_data_path = 'train.tsv'
test_data_path = 'valid.tsv'
output_path = 'output.txt'
stop_path = 'stopwords.txt'

In [8]:
train_df= pd.read_csv(train_data_path,header=None,sep='\t',quoting=3)
train_x= train_df[2]
train_y= train_df[1]
stop= pd.read_csv(stop_path,header=None)[0]
stop=stop.astype(str).tolist()

In [9]:
stemmer = PorterStemmer()

def preprocess_text(text):
    words = text.lower().split()
    # Filter out stopwords and apply stemming
    filtered_words = [stemmer.stem(word) for word in words if word not in stop]

    # Create unigrams (individual words)
    unigrams = filtered_words
    
    # Create bigrams (pairs of consecutive words)
    bigrams = ['_'.join(bigram) for bigram in ngrams(filtered_words, 2)]
    
    # Combine unigrams and bigrams
    return unigrams + bigrams

train_x= train_x.apply(preprocess_text)

In [10]:
label_to_index = {'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5}
index_to_label = {v: k for k, v in label_to_index.items()}


In [11]:
def create_feature_matrix(processed_texts, labels):
    vocabulary = set(word for text in processed_texts for word in text)
    vocab_size = len(vocabulary)
    
    word_to_index = {word: idx for idx, word in enumerate(vocabulary)}
    
    feature_matrix = np.zeros((len(processed_texts), vocab_size), dtype=int)
    class_count = np.zeros(len(set(labels)))

    for i, text in enumerate(processed_texts):
        for word in text:
            if word in word_to_index: 
                feature_matrix[i, word_to_index[word]] = 1
    
    encoded_labels = np.array([label_to_index[label] for label in labels])
    
    for label in labels:
        class_count[label_to_index[label]] += 1

    return feature_matrix, encoded_labels, class_count, word_to_index

X, Y, class_count, word_to_index= create_feature_matrix(train_x, train_y)

In [12]:
class BernoulliNaiveBayes:
    def __init__(self):
        self.class_priors = None
        self.feature_probs = None

    def fit(self, X, y):
        self.class_priors = np.log(np.bincount(y) / len(y))
        self.feature_probs = np.zeros((len(self.class_priors), X.shape[1]))
        
        for c in range(len(self.class_priors)):
            X_c = X[y == c]
            self.feature_probs[c] = (X_c.sum(axis=0) + 1) / (X_c.shape[0] + 6) 

    def predict(self, X):
        eps = 1e-10 
        log_feature_probs = np.log(self.feature_probs.T.clip(eps, 1 - eps))
        log_complement_probs = np.log((1 - self.feature_probs.T).clip(eps, 1 - eps))
        log_probs = X @ log_feature_probs + (1 - X) @ log_complement_probs + self.class_priors
        print(log_probs)
        return np.argmax(log_probs, axis=1)

model = BernoulliNaiveBayes()
model.fit(X, Y)

predictions = model.predict(X)

accuracy = np.mean(predictions == Y)
print(f'Accuracy: {accuracy * 100:.2f}%')

[[-234.6563753  -168.53958928 -186.15412741 -177.07305797 -182.58631384
  -187.43301648]
 [-297.87574589 -240.60017758 -246.81961062 -230.55592461 -243.61184523
  -249.97031967]
 [-277.75387173 -218.57072614 -227.08581655 -219.7832875  -212.71950027
  -227.62911733]
 ...
 [-312.60427138 -261.94892493 -266.80813014 -246.02656819 -265.54019849
  -267.67440222]
 [-214.57233623 -150.22859713 -166.82539533 -157.69953609 -156.81485386
  -164.91866451]
 [-310.08706005 -279.19541328 -288.19838228 -281.48433443 -284.00422878
  -285.46804562]]
Accuracy: 70.80%


In [13]:
predicted_labels = [index_to_label[pred] for pred in predictions]
print(predicted_labels)
print(len(predicted_labels))
with open(output_path, 'w') as f:
    for label in predicted_labels:
        f.write(f"{label}\n")

['false', 'half-true', 'mostly-true', 'false', 'half-true', 'true', 'half-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'false', 'mostly-true', 'half-true', 'half-true', 'true', 'barely-true', 'half-true', 'mostly-true', 'false', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'false', 'mostly-true', 'half-true', 'false', 'mostly-true', 'half-true', 'half-true', 'false', 'mostly-true', 'mostly-true', 'false', 'half-true', 'true', 'half-true', 'mostly-true', 'false', 'half-true', 'false', 'half-true', 'false', 'half-true', 'false', 'false', 'false', 'false', 'barely-true', 'half-true', 'false', 'true', 'half-true', 'half-true', 'half-true', 'barely-true', 'false', 'mostly-true', 'false', 'mostly-true', 'true', 'mostly-true', 'false', 'half-true', 'true', 'false', 'barely-true', 'half-true', 'true', 'half-true', 'mostly-true', 'half-true', 'mostly-true', 'true', 'false', 'half-true', 'false', 'half-true', 'half-true', 'half-true', 'false', 'false', '

In [14]:
file= 'checker_files/bernoulli_bigrams_probas_train.npy'
given= np.load(file)
print(np.log(given))
argmax_indices = np.argmax(given, axis=1)
argmax_indices= [index_to_label[z] for z in argmax_indices]
print(argmax_indices)
print(len(argmax_indices))
print(argmax_indices==predicted_labels)

[[-234.65755524 -168.54008841 -186.15472891 -177.07352778 -182.58682107
  -187.43360872]
 [-297.87692583 -240.60067671 -246.82021212 -230.55639442 -243.61235246
  -249.97091191]
 [-277.75505167 -218.57122527 -227.08641805 -219.78375731 -212.7200075
  -227.62970957]
 ...
 [-312.60545132 -261.94942406 -266.80873164 -246.02703801 -265.54070572
  -267.67499447]
 [-214.57351617 -150.22909626 -166.82599683 -157.7000059  -156.81536108
  -164.91925675]
 [-310.08823999 -279.1959124  -288.19898379 -281.48480424 -284.00473601
  -285.46863786]]
['false', 'half-true', 'mostly-true', 'false', 'half-true', 'true', 'half-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'false', 'mostly-true', 'half-true', 'half-true', 'true', 'barely-true', 'half-true', 'mostly-true', 'false', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'false', 'mostly-true', 'half-true', 'false', 'mostly-true', 'half-true', 'half-true', 'false', 'mostly-true', 'mostly-true', 'false', 'half-true

In [15]:
test_df= pd.read_csv(test_data_path,header=None,sep='\t',quoting=3)
test_x= train_df[2]
test_y= train_df[1]

In [16]:
def create_test_feature_matrix(processed_texts, word_to_index):
    vocab_size = len(word_to_index)
    
    # Initialize feature matrix for test data
    feature_matrix = np.zeros((len(processed_texts), vocab_size), dtype=int)
    
    for i, text in enumerate(processed_texts):
        for word in text:
            if word in word_to_index:  # Only include words from the training vocabulary
                feature_matrix[i, word_to_index[word]] = 1
                
    return feature_matrix

test_x= test_x.apply(preprocess_text)
X_test = create_test_feature_matrix(test_x, word_to_index)

test_predictions = model.predict(X_test)
test_predictions = [index_to_label[pred] for pred in test_predictions]
test_accuracy = np.mean(test_predictions == test_y)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


[[-234.6563753  -168.53958928 -186.15412741 -177.07305797 -182.58631384
  -187.43301648]
 [-297.87574589 -240.60017758 -246.81961062 -230.55592461 -243.61184523
  -249.97031967]
 [-277.75387173 -218.57072614 -227.08581655 -219.7832875  -212.71950027
  -227.62911733]
 ...
 [-312.60427138 -261.94892493 -266.80813014 -246.02656819 -265.54019849
  -267.67440222]
 [-214.57233623 -150.22859713 -166.82539533 -157.69953609 -156.81485386
  -164.91866451]
 [-310.08706005 -279.19541328 -288.19838228 -281.48433443 -284.00422878
  -285.46804562]]
Test Accuracy: 70.80%


In [17]:
print(test_predictions)

['false', 'half-true', 'mostly-true', 'false', 'half-true', 'true', 'half-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'false', 'mostly-true', 'half-true', 'half-true', 'true', 'barely-true', 'half-true', 'mostly-true', 'false', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'false', 'mostly-true', 'half-true', 'false', 'mostly-true', 'half-true', 'half-true', 'false', 'mostly-true', 'mostly-true', 'false', 'half-true', 'true', 'half-true', 'mostly-true', 'false', 'half-true', 'false', 'half-true', 'false', 'half-true', 'false', 'false', 'false', 'false', 'barely-true', 'half-true', 'false', 'true', 'half-true', 'half-true', 'half-true', 'barely-true', 'false', 'mostly-true', 'false', 'mostly-true', 'true', 'mostly-true', 'false', 'half-true', 'true', 'false', 'barely-true', 'half-true', 'true', 'half-true', 'mostly-true', 'half-true', 'mostly-true', 'true', 'false', 'half-true', 'false', 'half-true', 'half-true', 'half-true', 'false', 'false', '

In [18]:
print(test_y)

0              false
1          half-true
2        mostly-true
3              false
4          half-true
            ...     
10264    mostly-true
10265    mostly-true
10266      half-true
10267          false
10268     pants-fire
Name: 1, Length: 10269, dtype: object


In [19]:
valfile= 'checker_files/bernoulli_bigrams_probas_test.npy'
givenval= np.load(valfile)
print(np.log(givenval))
argmax_indices2 = np.argmax(given, axis=1)
argmax_indices2= [index_to_label[z] for z in argmax_indices2]
print(argmax_indices2)
print(len(argmax_indices2))
print(argmax_indices2==test_predictions)

[[-160.79293913  -95.90739344 -107.19692992  -92.78167981  -96.11893386
  -104.6736985 ]
 [-184.0801682  -121.59416207 -131.50170457 -120.12565172 -123.9183019
  -129.01961855]
 [-233.23146395 -170.04035237 -180.14743209 -170.12778482 -171.75672615
  -178.81101364]
 ...
 [-204.34289254 -140.15188152 -150.95931058 -137.02217872 -141.75947574
  -146.17643775]
 [-269.18131856 -207.48618215 -215.78136502 -204.68733544 -210.75904492
  -216.49066708]
 [-204.84791901 -133.10258626 -148.43386655 -134.50802796 -138.65791188
  -147.41519882]]
['false', 'half-true', 'mostly-true', 'false', 'half-true', 'true', 'half-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'false', 'mostly-true', 'half-true', 'half-true', 'true', 'barely-true', 'half-true', 'mostly-true', 'false', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'false', 'mostly-true', 'half-true', 'false', 'mostly-true', 'half-true', 'half-true', 'false', 'mostly-true', 'mostly-true', 'false', 'half-true